## Лабораторная 4

1. Сгенерировать набор данных при помощи make_classification из sklearn.
2. Построить нейронную сеть для решения задачи классификации (можете использовать лабораторную 3).
3. Исследовать влияние различных классификаторов на качество модели (SGD, Adam, RMSProp), записать выводы.
4. Добавить к модели метод регуляризации Dropout и повторить пункт 3 уже с ним.

*Дополнительное задание:
Повторить пункты 2-4 для набора данных MNIST.*

In [37]:
from sklearn.datasets import make_classification
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

In [38]:
X_gen, y_gen = make_classification(
    n_samples=1000,
    n_features=20,
    n_informative=10,
    n_redundant=10,
    n_classes=2,
    random_state=42
)

print(X_gen.shape, y_gen.shape)

(1000, 20) (1000,)


In [39]:
model = Sequential([
    Input(shape=(X_gen.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential_20"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_60 (Dense)                │ (None, 64)             │         1,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_61 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_62 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,457 (13.50 KB)

 Trainable params: 3,457 (13.50 KB)

 Non-trainable params: 0 (0.00 B)

In [40]:
X_train, X_test, y_train, y_test = train_test_split(
    X_gen, y_gen,
    test_size=0.2,
    random_state=42
)

print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_train: {y_train.shape}")
print(f"y_test: {y_test.shape}")

X_train: (800, 20)
X_test: (200, 20)
y_train: (800,)
y_test: (200,)


In [41]:
optimizers = {
    'SGD': SGD(learning_rate=0.01),
    'Adam': Adam(learning_rate=0.001),
    'RMSprop': RMSprop(learning_rate=0.001)
}

results = {}

for opt_name, optimizer in optimizers.items():
    print(f"\nTraining with {opt_name}:")

    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    history = model.fit(
        X_train, y_train,
        epochs=15,
        batch_size=32,
        verbose=0,
        validation_data=(X_test, y_test)
    )

    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Loss with {opt_name}: {loss:.4f}")
    print(f"Test Accuracy with {opt_name}: {accuracy:.4f}")
    results[opt_name] = {'loss': loss, 'accuracy': accuracy}


Training with SGD:
Test Loss with SGD: 0.2291
Test Accuracy with SGD: 0.9300

Training with Adam:
Test Loss with Adam: 0.1241
Test Accuracy with Adam: 0.9450

Training with RMSprop:
Test Loss with RMSprop: 0.1216
Test Accuracy with RMSprop: 0.9400


In [42]:
model_dropout = Sequential([
    Input(shape=(X_gen.shape[1],)),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model_dropout.summary()


Model: "sequential_21"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_63 (Dense)                │ (None, 64)             │         1,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_64 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_65 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,457 (13.50 KB)

 Trainable params: 3,457 (13.50 KB)

 Non-trainable params: 0 (0.00 B)

In [43]:
optimizers = {
    'SGD': SGD(learning_rate=0.01),
    'Adam': Adam(learning_rate=0.001),
    'RMSprop': RMSprop(learning_rate=0.001)
}

results = {}

for opt_name, optimizer in optimizers.items():
    print(f"\nTraining with {opt_name}:")

    model_dropout.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    history = model_dropout.fit(
        X_train, y_train,
        epochs=15,
        batch_size=32,
        verbose=0,
        validation_data=(X_test, y_test)
    )

    loss, accuracy = model_dropout.evaluate(X_test, y_test, verbose=0)
    print(f"Test Loss with {opt_name}: {loss:.4f}")
    print(f"Test Accuracy with {opt_name}: {accuracy:.4f}")
    results[opt_name] = {'loss': loss, 'accuracy': accuracy}


Training with SGD:
Test Loss with SGD: 0.3577
Test Accuracy with SGD: 0.8800

Training with Adam:
Test Loss with Adam: 0.1783
Test Accuracy with Adam: 0.9500

Training with RMSprop:
Test Loss with RMSprop: 0.1353
Test Accuracy with RMSprop: 0.9550


Вот переписанный вывод **в той же структуре и стиле**, но с подстановкой **твоих новых чисел**.

---

## **Вывод**

### Без Dropout

* **SGD** — Loss **0.2291**, Accuracy **0.9300**
* **Adam** — Loss **0.1241**, Accuracy **0.9450**
* **RMSprop** — Loss **0.1216**, Accuracy **0.9400**

Adam даёт лучшую точность.
RMSprop — самый маленький loss.
SGD — наихудший по показателям.

---

### С Dropout

* **SGD** — Loss **0.3577**, Accuracy **0.8800**
* **Adam** — Loss **0.1783**, Accuracy **0.9500**
* **RMSprop** — Loss **0.1353**, Accuracy **0.9550**

RMSprop - лучший: минимальный loss и максимальная точность.
Adam — стабильно хороший, но чуть хуже RMSprop.
SGD — наихудший по показателям.

---

* **Без Dropout** лучше всех показывает себя **Adam**.
* **С Dropout** лучший — **RMSprop**.
* **SGD** дает. наихудшую точность.
* Dropout улучшает Adam и RMSprop, но сильно ухудшает SGD.
